<a href="https://colab.research.google.com/github/suhaodatascichem/dashboard-streamlit/blob/main/aminoinsightsDEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
# Load your dataset
# Replace 'your_data.csv' with your actual data file
url = 'https://github.com/suhaodatascichem/dashboard-streamlit/raw/refs/heads/main/Soybean%20meal.xlsx'
df = pd.read_excel(url, engine="openpyxl")

In [ ]:
# App title
st.title("Customer Data Dashboard")

    # ---- Sidebar Filters ----
st.sidebar.header("Filters")

    # Ensure "Date" column is in datetime format
if "Release Date" in df.columns:
    df["Release Date"] = pd.to_datetime(df["Release Date"])

    # Date Range Filter
    min_date, max_date = df["Release Date"].min(), df["Release Date"].max()
    start_date, end_date = st.sidebar.date_input("Select Date Range", [min_date, max_date], min_value=min_date, max_value=max_date)

    # Customer Filter (Dropdown with Multi-selection)
if "Customer" in df.columns:
    customers = df["Customer"].unique().tolist()
  # Add "Select All" option at the beginning
    all_customers_option = ["Select All"] + customers
  # Display multiselect with "Select All" option
    selected_customers = st.sidebar.multiselect("Select Customer(s)", all_customers_option, default=["Select All"])

    # If "Select All" is chosen, use all customers
    if "Select All" in selected_customers:
        selected_customers = customers  # Reset to all customers

# Material Type Filter (Dropdown with Multi-selection)
if "Material" in df.columns:
    material_types = df["Material"].unique()
    selected_materials = st.sidebar.multiselect("Select Material Type(s)", material_types, default=material_types)

# ---- Apply Filters ----
filtered_df = df.copy()

if "Release Date" in df.columns:
    filtered_df = filtered_df[(filtered_df["Release Date"] >= pd.to_datetime(start_date)) & (filtered_df["Release Date"] <= pd.to_datetime(end_date))]
if "Customer" in df.columns:
    filtered_df = filtered_df[filtered_df["Customer"].isin(selected_customers)]
if "Material" in df.columns:
    filtered_df = filtered_df[filtered_df["Material"].isin(selected_materials)]

# ---- Display Filtered Data ----
st.write("### Filtered Data Preview")
st.write(f"🔍 **Total Records After Filtering:** {filtered_df.shape[0]}")
st.dataframe(filtered_df)


# ---- Charts Section ----
st.write("## 📊 Data Visualizations")

# Predefined Charts
if "CP" in filtered_df.columns and "DM" in filtered_df.columns:
    fig1 = px.line(filtered_df, x="CP", y="DM", title="📈 DM Over CP")
    st.plotly_chart(fig1)

if "CP" in filtered_df.columns and "NE_S" in filtered_df.columns:
    fig2 = px.bar(filtered_df, x="CP", y="NE_S", title="🏢 NE_S over CP")
    st.plotly_chart(fig2)

if "CP" in filtered_df.columns and "AME_KCAL" in filtered_df.columns:
    fig3 = px.bar(filtered_df, x="CP", y="AME_KCAL", title="📦 AME_KCAL over CP")
    st.plotly_chart(fig3)

if "CP" in filtered_df.columns and "NH3" in filtered_df.columns:
    fig4 = px.scatter(filtered_df, x="CP", y="NH3", title="🔍 NH3 vs CP")
    st.plotly_chart(fig4)

# ---- Custom Chart Selection ----
st.write("### 🛠 Create Your Own Chart")

# Column selection for custom charts
available_columns = list(filtered_df.select_dtypes(include=["number", "object"]).columns)
x_axis = st.selectbox("Select X-Axis", available_columns)
y_axis = st.selectbox("Select Y-Axis", available_columns)

chart_type = st.radio("Select Chart Type", ["Scatter", "Bar","Histogram"])

# Generate custom chart
if st.button("Generate Chart"):
    if chart_type == "Bar":
        y_axis = st.sidebar.selectbox("Select Y-Axis", available_columns)
        fig = px.bar(df, x=x_axis, y=y_axis, title=f"📊 {y_axis} by {x_axis}")

    elif chart_type == "Scatter":
        y_axis = st.sidebar.selectbox("Select Y-Axis", available_columns)
        fig = px.scatter(df, x=x_axis, y=y_axis, title=f"📊 {x_axis} vs {y_axis}")

    elif chart_type == "Histogram":
        fig = px.histogram(df, x=x_axis, title=f"📊 Histogram of {x_axis}")

    st.plotly_chart(fig_custom)
